In [419]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the dataset for recommenders

In [420]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df['term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df['length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df['rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df['room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df['n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df['weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df['weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(interactions_df.head(15))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

$$
    r_{u, i} = f(user\_features, item\_features)
$$

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several versions of the prepare_users_df method and test which works best in your recommender.

In [421]:
from features.user_features import UserFeatures

def prepare_users_df(interactions_df):

    users_df = interactions_df.copy()

    # Declare the user features list
    user_features = list()

    # Define helper function to add a feature to dataframe and features list
    def apply(func):
        feature_names, df_result = func(users_df)
        user_features.extend(feature_names)
        return df_result

    # Apply user features functions
    users_df = apply(UserFeatures.most_popular_user_features_one_hot_encoding)
    users_df = apply(UserFeatures.features_propability_distribution)
    users_df = apply(UserFeatures.features_averange)
    users_df = apply(UserFeatures.most_popular_reservation_similarity)

    return users_df, user_features
    

users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))

['most_popular_term', 'most_popular_length_of_stay_bucket', 'most_popular_rate_plan', 'most_popular_room_segment', 'most_popular_n_people_bucket', 'most_popular_weekend_stay', 'term_propability_distribution', 'length_of_stay_bucket_propability_distribution', 'rate_plan_propability_distribution', 'room_segment_propability_distribution', 'n_people_bucket_propability_distribution', 'weekend_stay_propability_distribution', 'length_of_stay_averange', 'room_segment_averange', 'n_people_averange', 'most_popular_similarity']


,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay,most_popular_term,most_popular_length_of_stay_bucket,...,term_propability_distribution,length_of_stay_bucket_propability_distribution,rate_plan_propability_distribution,room_segment_propability_distribution,n_people_bucket_propability_distribution,weekend_stay_propability_distribution,length_of_stay_averange,room_segment_averange,n_people_averange,most_popular_similarity
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True,0,1,...,0.095101,0.593660,0.482447,0.161056,0.148342,0.691512,3.727273,223.636364,2.590909,0.35
42,50,31,OffSeason,[2-3],Standard,[160-260],[5-inf],True,1,1,...,0.438499,0.593660,0.482447,0.793555,0.148342,0.691512,2.760870,253.478261,3.847826,0.75
47,50,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True,1,1,...,0.438499,0.593660,0.482447,0.161056,0.148342,0.691512,2.760870,253.478261,3.847826,0.55
83,1,32,OffSeason,[2-3],Standard,[160-260],[2-2],True,1,1,...,0.438499,0.593660,0.482447,0.793555,0.343467,0.691512,3.727273,223.636364,2.590909,0.75
84,1,32,OffSeason,[2-3],Standard,[160-260],[2-2],True,1,1,...,0.438499,0.593660,0.482447,0.793555,0.343467,0.691512,3.727273,223.636364,2.590909,0.75
318,1,60,WinterVacation,[2-3],Nonref,[160-260],[2-2],True,0,1,...,0.095101,0.593660,0.517553,0.793555,0.343467,0.691512,3.727273,223.636364,2.590909,0.65
333,706,114,WinterVacation,[2-3],Standard,[160-260],[1-1],False,0,1,...,0.095101,0.593660,0.482447,0.793555,0.068078,0.308488,3.285714,216.071429,3.228916,0.40
387,706,106,OffSeason,[8-inf],Standard,[160-260],[1-1],True,1,0,...,0.438499,0.018797,0.482447,0.793555,0.068078,0.691512,3.285714,216.071429,3.228916,0.55
439,706,17,OffSeason,[2-3],Standard,[160-260],[1-1],True,1,1,...,0.438499,0.593660,0.482447,0.793555,0.068078,0.691512,3.285714,216.071429,3.228916,0.75
456,706,143,OffSeason,[0-1],Standard,[160-260],[1-1],False,1,0,...,0.438499,0.146254,0.482447,0.793555,0.068078,0.308488,3.285714,216.071429,3.228916,0.40


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into one-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features, for instance based on numerical min and max values in buckets used as features.

In [422]:
from features.item_features import ItemFeatures

def prepare_items_df(interactions_df):

    items_df = interactions_df.copy()

    # Declare the item features list
    item_features = list()

    # Define helper function to add a feature to dataframe and features list
    def apply(func):
        feature_names, df_result = func(items_df)
        item_features.extend(feature_names)
        return df_result

    # Apply item features functions
    apply(ItemFeatures.is_cheapest_and_is_most_expensive)
    # apply(ItemFeatures.most_frequent_number_of_people)
    apply(ItemFeatures.is_item_most_popular)
    
    # Write your code here
    
    return items_df, item_features


items_df, item_features = prepare_items_df(interactions_df)

print(item_features)

display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))

['is_cheapest', 'is_most_expensive', 'is_most_popular']


,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay,is_cheapest,is_most_expensive,is_most_popular
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True,0,0,0
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True,0,0,0
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False,0,0,0
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True,0,0,0
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True,1,0,0
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True,0,0,0
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True,0,0,0
44,52,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True,0,0,0
47,50,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True,0,0,0
51,58,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True,0,0,0


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object as self.users_df for later use in the recommend method. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real ("positive") interactions and equal to 0 for those newly added "negative" interactions. Generate several negative interactions per every positive interaction (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, LightGBM, Neural Networks or any model of your choice.

In [438]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        # Prepare users_df and items_df
        
        users_df, user_features = prepare_users_df(interactions_df)
        users_df = users_df.loc[:, ['user_id'] + user_features]
        
        self.users_df = users_df
        self.user_features = user_features
        
        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]
        
        # Generate negative interactions
        
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1
        
        negative_interactions = []

        # Generate tuples (user_id, item_id, 0) for pairs (user_id, item_id) which do not
        # appear in the interactions_df and add those tuples to the list negative_interactions.
        # Generate self.n_neg_per_pos * len(interactions_df) negative interactions 
        # (self.n_neg_per_pos per one positive).
        # Make sure the code is efficient and runs fast, otherwise you will not be able to properly tune your model.

        self.n_neg_per_pos = int(self.n_neg_per_pos)

        def generate_negative_interactions(interactions_df):
            interactions_pivot = interactions_df.pivot_table(index='user_id', columns='item_id', aggfunc=len, fill_value=0).fillna(0)
            interactions_pivot['sum_of_interactions'] = interactions_pivot.sum(axis=1)

            for user_row in interactions_pivot.iterrows():
                user_id = user_row[0]
                sum_of_interactions = user_row[1]['sum_of_interactions'][0]
                available_item_ids = list(user_row[1]['interacted'].loc[user_row[1]['interacted'] == 0].index)
                random_choice = self.rng.choice(available_item_ids, size=sum_of_interactions * self.n_neg_per_pos, replace=False)
                tuple_list = list(map(lambda item_id: (user_id, item_id, 0), random_choice))

                negative_interactions.extend(tuple_list)

        generate_negative_interactions(interactions_df)
        
        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        
        # Get the input data for the model
        
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])

        interactions_df = interactions_df.fillna(0)
        
        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values

        self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
        
        # Prepare users_df and items_df
        # For users_df you need to merge user features from self.users_df to users_df 
        # (the users for which you generate recommendations).
        # Note that for users who were not in the hotel before (which is true for most users)
        # there will be no features in self.users_df. For such users you can initialize their features
        # with all zeros (for instance with fillna(0)), but you can also try to use average feature
        # values from self.users_df (this way you would trear a new user as an average user).
        # For items you have to apply the prepare_items_df method to items_df.

        users_df = pd.merge(users_df, self.users_df.loc[:, ['user_id'] + self.user_features], on=['user_id'], how='left').fillna(value=self.users_df.mode().iloc[0].to_dict())
        user_features = self.user_features
        items_df, item_features = prepare_items_df(items_df)
        
        # Score the items
        
        for ix, user in users_df.iterrows():

            prepared_user = user.to_frame().transpose()

            recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
            
            # Create a Carthesian product of user and items from items_df
            # Cartesian product of user and items from items_df. 
            cartesian_product = pd.merge(prepared_user, items_df, how='cross')

            # Use self.model.predict method to calculate scores for all records in the just created DataFrame
            # of users and items

            predictions = self.model.predict(cartesian_product.loc[:, user_features + item_features].values)
            recommendations = pd.DataFrame({'item_id': cartesian_product['item_id'], 'score': predictions})
            
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.

            recommendations = recommendations.drop_duplicates(subset=['item_id']).sort_values(by='score', ascending=False).head(n_recommendations)
            recommendations['score']

            chosen_ids = recommendations.item_id.values
            scores = recommendations.loc[:, ['item_id', 'score']].set_index('item_id')
            scores['score'] = scores['score']
            scores = scores['score'].to_dict()
            
            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score':  scores[item_id]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df
    
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)    

# Quick test of the recommender

In [439]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [440]:
# Fit method
cb_user_item_recommender = RandomForestCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [426]:
# Recommender method
recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations)

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,51,1.0,OffSeason,[2-3],Nonref,[160-260],[3-4],True
1,1.0,0,0.808249,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
2,1.0,473,0.808249,NewYear,[4-7],Nonref,[160-260],[3-4],True
3,1.0,464,0.808249,Christmas,[2-3],Standard,[160-260],[3-4],True
4,1.0,465,0.808249,NewYear,[2-3],Nonref,[160-260],[2-2],False
...,...,...,...,...,...,...,...,...,...
255,5.0,627,0.417221,LowSeason,[0-1],Nonref,[0-160],[2-2],True
256,5.0,626,0.417221,LowSeason,[4-7],Nonref,[0-160],[1-1],False
257,5.0,622,0.417221,OffSeason,[0-1],Nonref,[0-160],[1-1],False
258,5.0,621,0.417221,HighSeason,[8-inf],Nonref,[0-160],[2-2],True


# Tuning method

In [427]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [428]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(results)
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [429]:
# Best parameters:
# {'n_neg_per_pos': 9.0}

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 5, 10, 1)
}

# param_space = {
#     'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
# }

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=5, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 5/5 [04:57<00:00, 59.58s/trial, best loss: -0.01396529834955565]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.003055,0.003055,0.003055,0.004073,0.003055,0.003055,0.003055,0.003399


Best parameters:
{'n_neg_per_pos': 8.0}


In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [456]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 8, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=3, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 3/3 [31:13<00:00, 624.64s/trial, best loss: -0.013542107490478206]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.002716,0.003734,0.004413,0.008147,0.002716,0.003225,0.003517,0.004704


Best parameters:
{'max_depth': 8.0, 'min_samples_split': 18.0, 'n_estimators': 255.0, 'n_neg_per_pos': 9.0}


In [457]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=2, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  0%|          | 0/2 [04:22<?, ?trial/s, best loss=?]
Best parameters:
None


Traceback (most recent call last):
  File "C:\Users\Marcin\AppData\Local\Temp\ipykernel_20648\3138901096.py", line 33, in tune_recommender
    best_param_set = fmin(loss, space=param_space, algo=tpe.suggest,
  File "C:\Users\Marcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\hyperopt\fmin.py", line 540, in fmin
    return trials.fmin(
  File "C:\Users\Marcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\hyperopt\base.py", line 671, in fmin
    return fmin(
  File "C:\Users\Marcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\hyperopt\fmin.py", line 586, in fmin
    rval.exhaust()
  File "C:\Users\Marcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\hyperopt\fmin.py", line 364, in exhau

# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [ ]:
# cb_user_item_recommender = LinearRegressionCBUIRecommender(
#     **{'n_neg_per_pos': 9})

cb_user_item_recommender = XGBoostCBUIRecommender(
    **{'n_neg_per_pos': 9,
    'n_estimators': 267,
    'max_depth': 8,
    'min_samples_split': 24,
    'learning_rate': 0.007})

# Give the name of your recommender in the line below
# linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
#     cb_user_item_recommender, interactions_df, items_df))]


linear_cbui_tts_results = [['XGBoostCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_cbui_tts_results)

In [454]:
cb_user_item_recommender = LinearRegressionCBUIRecommender(
    **{'n_neg_per_pos': 9})

# Give the name of your recommender in the line below
linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.003055,0.004073,0.004752,0.008147,0.003055,0.003564,0.003857,0.00493


In [458]:
cb_user_item_recommender = RandomForestCBUIRecommender(
    **{'n_neg_per_pos': 9,
    'n_estimators': 255,
    'max_depth': 8,
    'min_samples_split': 18})

random_forest_cbui_tts_results = [['RandomForestCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

random_forest_cbui_tts_results = pd.DataFrame(
    random_forest_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(random_forest_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.002716,0.003734,0.004413,0.008486,0.002716,0.003225,0.003517,0.004811


In [ ]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(amazon_tts_results)

In [455]:
tts_results = pd.concat([linear_cbui_tts_results, random_forest_cbui_tts_results, amazon_tts_results]).reset_index(drop=True)
display(tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.003055,0.004073,0.004752,0.008147,0.003055,0.003564,0.003857,0.004930
1,AmazonRecommender,0.044128,0.118805,0.160896,0.223693,0.044128,0.086755,0.104347,0.124505
